In [2]:
import pandas as pd
import ast
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random


ModuleNotFoundError: No module named 'tensorflow'

In [41]:
strike_data = pd.read_csv("total_strike_data.csv")

In [42]:
strike_data.head()

,data,label
0,"[[29971.0, 29980.0, 29988.0, 29997.0, 30005.0,...",jab
1,"[[37278.0, 37286.0, 37294.0, 37302.0, 37310.0,...",upper
2,"[[43689.0, 43697.0, 43705.0, 43713.0, 43721.0,...",upper
3,"[[49913.0, 49922.0, 49931.0, 49938.0, 49947.0,...",hook
4,"[[56391.0, 56399.0, 56407.0, 56416.0, 56424.0,...",upper


In [43]:
# Change type of punch to numbers (jab - 0, upper - 1, hook - 2)
punches = ['jab', 'upper', 'hook']
for i in range(len(strike_data['label'])):
    strike_data['label'][i] = punches.index(strike_data['label'][i])

In [44]:
# Transform string list to actual lists seperated into different readings
list_strike_data = [ ast.literal_eval(strike_data['data'][i]) for i in range(len(strike_data['data'])) ]
time = [ list_strike_data[i][0] for i in range(len(strike_data['data'])) ] 
accx = [ list_strike_data[i][1] for i in range(len(strike_data['data'])) ] 
accy = [ list_strike_data[i][2] for i in range(len(strike_data['data'])) ] 
accz = [ list_strike_data[i][3] for i in range(len(strike_data['data'])) ] 
magx = [ list_strike_data[i][4] for i in range(len(strike_data['data'])) ] 
magy = [ list_strike_data[i][5] for i in range(len(strike_data['data'])) ] 
magz = [ list_strike_data[i][6] for i in range(len(strike_data['data'])) ] 
gyrox = [ list_strike_data[i][7] for i in range(len(strike_data['data'])) ] 
gyroy = [ list_strike_data[i][8] for i in range(len(strike_data['data'])) ] 
gyroz = [ list_strike_data[i][9] for i in range(len(strike_data['data'])) ] 
strikes = strike_data['label'].to_list()

# Base time to 0
for i in range(len(time)):
    min_time = min(time[i])
    for j in range(len(time[i])):
        time[i][j] -= min_time

data_list = [time, accx, accy, accz, magx, magy, magz, gyrox, gyroy, gyroz, strikes]
columns =['time', 'accx', 'accy', 'accz', 'magx', 'magy', 'magz', 'gyrox', 'gyroy', 'gyroz', 'strikes']

In [45]:
# Find data samples that have less than 110 readings
list_lengths = [ len(time[i]) for i in range(len(time)) ]

bad = []

for i in range(len(list_lengths)):
    if(list_lengths[i]<110):
        bad.append(i)
bad


[24, 73, 194, 202, 207, 243, 442, 480, 493, 544, 655, 758, 805, 847, 928, 1012]

In [46]:
# Look at bad indexes and pop them out of main data
for index in bad[-1::-1]:
    for i in range(len(columns)):
        data_list[i].pop(index)



In [47]:
# Chop off data from the front to make all samples equal length
for reading in data_list:
    for sample in reading:
        if(type(sample) == int):
            continue
        while(len(sample) > 110):
            sample.pop(0)

In [48]:
# Transform data_list so each sample point in one long list with time + all IMU readings
new_data_list = []

for i in range(len(data_list[0])):
    new_data_list.append([])
    for j in range(len(data_list)-1):
        new_data_list[i] += data_list[j][i]

#len(new_data_list[0])
new_data_list_strikes = data_list[-1]

In [49]:
tf.compat.v1.disable_eager_execution()

x = tf.compat.v1.placeholder(tf.float32,shape=[None, 1100])
W = tf.Variable(tf.zeros([1100, 3]))
b = tf.Variable(tf.zeros([3]))
# Create the Graph
y = tf.matmul(x,W) + b 
y_true = tf.compat.v1.placeholder(tf.float32,[None, 3])

In [50]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y))

In [51]:
optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.5)

In [52]:
train = optimizer.minimize(cross_entropy)

In [53]:
init = tf.compat.v1.global_variables_initializer()

In [54]:
class strike:
    # data = List[List] - [ [time, accx, ..., magz], [time,accx ..], ...  ]
    def __init__(self, data, results, train_per = 0.67):
        self.data = data
        self.results = results
        self.index = 0
        self.data_len = len(self.data)

        self.X_train, self.X_test, self.y_train, self.y_test = self.train_test_split(train_per)

    def next_batch(self, amount):
        new_index = self.index+amount
        if(new_index > len(self.X_train)):
            new_index = new_index - len(self.X_train)
            batch_x = self.X_train[self.index:] + self.X_train[0:new_index]
            batch_y = self.y_train[self.index:] + self.y_train[0:new_index]
            self.index =new_index

            return [batch_x, batch_y]

        return_data = [ self.X_train[self.index:new_index], self.y_train[self.index:new_index] ]
        self.index = new_index
        return return_data

    def train_test_split(self, train_percent):
        test_index = []
        train_index = []
        train_amount = round(self.data_len*train_percent)
        #create random train indexes
        for _ in range(train_amount):
            index = 0
            while True:
                index = random.randint(0,self.data_len-1)
                if(index not in test_index):
                    break
            train_index.append(index)    
        #create remaining test indexes
        test_index = list(range(0,self.data_len))
        for index in train_index:
            if index in test_index:
                test_index.remove(index)

        #seperating training and test data
        X_train = []
        X_test =  []
        y_train = [ self.results[index] for index in train_index ]
        y_test =  [ self.results[index] for index in test_index ]

        for index in train_index:
            X_train.append(self.data[index])
        for index in test_index:
            X_test.append(self.data[index])

        return [X_train, X_test, y_train, y_test]


In [55]:
strike = strike(data_list, new_data_list_strikes)

In [60]:
with tf.compat.v1.Session() as sess:
    sess.run(init)
    
    # Train the model for 1000 steps on the training set
    # Using built in batch feeder from mnist for convenience
    
    for step in range(1000):

        batch_x , batch_y = strike.next_batch(100)
        
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y})
        
    # Test the Train Model
    matches = tf.equal(tf.argmax(y,1),tf.argmax(y_true,1))
    
    acc = tf.reduce_mean(tf.cast(matches,tf.float32))
    
    print(sess.run(acc,feed_dict={x:strike.X_test,y_true:y_test}))

step:0


ValueError: Cannot feed value of shape (7, 1017, 110) for Tensor 'Placeholder_5:0', which has shape '(None, 1100)'

In [61]:
from tensorflow.examples.tutorials.mnist import input_data

ModuleNotFoundError: No module named 'tensorflow.examples'